In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import torch


In [26]:
import os

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True, google_api_key=GOOGLE_API_KEY)




In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

key = os.getenv("OPENAI_KEY")
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
llm = OpenAI(openai_api_key=key)

In [40]:
llm = Ollama(model="mistral",  callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),num_gpu=1, base_url="http://localhost:11434")

In [ ]:
modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/sid/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=key)


In [21]:
loader = DirectoryLoader("./data", glob="*.pdf", loader_cls=PyPDFLoader)
# loader = DirectoryLoader("/home/sid/Documents", glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)


91

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)

250

In [23]:
# import torch
# torch.cuda.empty_cache()
# print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
torch.cuda.empty_cache()
gc.collect()

15310

In [24]:
from langchain.vectorstores import Chroma
persist_directory = './db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()

In [25]:
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
dev

'cuda:0'

In [26]:
retriever = vectordb.as_retriever(search_kwargs={'k': 10})

In [27]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k =  7

In [28]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                                       weights=[0.3, 0.7])

In [29]:
ensemble_retriever.get_relevant_documents("How to focus on the task at hand?")

[Document(page_content='____________________________________________________', metadata={'page': 3, 'source': 'data/8.pdf'}),
 Document(page_content='embedded project component.  \n•Focus on experiential learning.  \n•Courses made student- centric \nrather than teacher -centric . \n•Students to take up real world \nproblems and focus on multi -\ndisciplinary and cross -disciplinary \nprojects . \n•Importance given to creativity, \ninnovation and development of \nentrepreneurship skills .', metadata={'page': 43, 'source': 'data/19.pdf'}),
 Document(page_content='____________________________________', metadata={'page': 6, 'source': 'data/8.pdf'}),
 Document(page_content='•40% of the courses to have \nembedded project component.  \n•Focus on experiential learning.  \n•Courses made student- centric \nrather than teacher -centric . \n•Students to take up real world \nproblems and focus on multi -\ndisciplinary and cross -disciplinary \nprojects . \n•Importance given to creativity, \ninnovat

In [30]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=ensemble_retriever,
                                  return_source_documents=True)
     

In [31]:
def process_llm_response(query):
    llm_response = qa_chain(query)
    return llm_response['result']
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [32]:
query = "How to apply for grade improvement?"
process_llm_response(query)
# qa_chain(query)

" To apply for grade improvement, you must register for the same course again during a subsequent Course Registration process. This course will be treated as another course taken by the student and no relaxation in the maximum number of credits a student can register during a semester will be given. However, you will have to pay the course fee for registering the course again. This option can only be availed once for a given course and is only available during regular semesters. Students at their graduating year or who are timed out are permitted to register for more than one 'Grade Improvement' course."

In [ ]:
def main():
    st.title("VIT QA System")
    user_input = st.text_area("Enter Query")

    if st.button("Get response"):
        result = process_llm_response(user_input)
        st.write("Response:")
        # make text bigger
        st.write(f"**{result}**")
main()